In [1]:
import pymongo


class MongoStorage:
    """Class for working with MongoDB"""

    db: pymongo.database.Database
    col: pymongo.collection.Collection

    def __init__(self, db: pymongo.database.Database, col: pymongo.collection.Collection):
        self.db = db
        self.col = col

    @classmethod
    def connect(cls, host: str, port=27017, db_name='wiki', col_name='articles'):
        db = pymongo.MongoClient(host, port, unicode_decode_error_handler='ignore')[db_name]
        return cls(
            db=db,
            col=db[col_name])

    def get_articles(self, count=0) -> list:
        return list(self.col.find({}).limit(count))

    def get_article(self, title) -> dict:
        doc = self.col.find_one({'title': title})
        return doc if doc else {}

    def get_articles_headings_texts(self, count=0) -> list:
        articles = self.get_articles(count)
        return [article['text']['Заголовок']['text'] for article in articles]


In [2]:
import psycopg2


class PostgresStorage:

    def __init__(self, conn):
        self.conn = conn
        self.cursor = conn.cursor()

    @staticmethod
    def connect(host, port=5432, user='postgres', password='password', dbname='habr'):
        return PostgresStorage(conn=psycopg2.connect(
            host=host, port=port, user=user, password=password, dbname=dbname)
        )

    def get_posts(self, count=0) -> list:
        if count > 0:
            self.cursor.execute('SELECT * FROM posts LIMIT %d' % count)
        else:
            self.cursor.execute('SELECT * FROM posts')
        posts = list(self.cursor.fetchall())
        return posts

    def get_posts_texts(self, count=0) -> list:
        posts_list = self.get_posts(count)
        return [post[2] for post in posts_list]


In [3]:
storage = PostgresStorage.connect(
            host='172.17.0.2')
posts = storage.get_posts_texts(1000)
posts[999]

' \n\nЭто не готовая инструкция, с большим количеством кода, а скорее описание алгоритма и результатов чего мы добились.\n\nИтак, не так давно у нас появился новый клиент. У него было несколько нетипичных для нас требований: использовать для конфигурирования серверов ansible, контент сайта хранится в git, каждый сайт находится на своей виртуальной машине. Все это не сулило ничего хорошего, так как совсем не укладывалось в стандартную схему «Клиент всегда прав!», и мы начали разрабатывать новую схему. Но обо всем по порядку. \n\nИсходные данные: есть клиент, у которого более 30 сайтов которые надо перенести на нашу площадку. Каждый сайт должен располагаться в отдельном контейнере (мы используем OpenVZ контейнеры). Используется только один внешний IP. Для конфигурирования серверов используется ansible. Для каждого сайта есть архив с конфигурационными файлами. Контент сайта находится в git. \n \nИ мы начали творить… Что у нас получилось, можно посмотреть под катом. Забегая вперед, скажу, 

In [4]:
import re


class Tokenizer:

    to_sentences = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s')
    remove_brackets = re.compile(r' \((.*?)\)')
    remove_punctuation = re.compile(r'[^a-zA-Zа-яА-Я ]')

    @classmethod
    def tokenize(cls, text: str, remove_punctuation=True, remove_brackets=True, remove_endings=True) -> list:
        buf = text.split('\n')
        buf = [item for item in buf if item]
        sentences = []
        if not remove_endings:
            for sentence in cls.to_sentences.split(' '.join(buf)):
                buf_list = [item for item in sentence.split('!') if item]
                for item in buf_list:
                    if item[-1] == '.' or item[-1] == '?':
                        sentences.append(item.lower())
                    else:
                        sentences.append(item.lower() + '!')
        else:
            sentences = [sentence[:-1].lower() for sentence in cls.to_sentences.split(' '.join(buf)) if sentence[:-1]]
        if remove_brackets:
            sentences = [cls.remove_brackets.sub('', sentence) for sentence in sentences]
        if remove_punctuation:
            return [cls.remove_punctuation.sub('', sentence) for sentence in sentences]
        return sentences


class TextProcessor:

    tokenizer = Tokenizer()

    @classmethod
    def __get_sentences_list(cls, text_list: list, remove_brackets=True, remove_endings=True) -> list:
        sentences_list = []
        for text in text_list:
            sentences_list += cls.tokenizer.tokenize(
                text=text,
                remove_brackets=remove_brackets,
                remove_endings=remove_endings)
        return sentences_list

    @classmethod
    def process_text_list(cls, text_list: list, window_size=1, remove_brackets=True, remove_endings=True) -> str:
        text = ''
        sentences_list = cls.__get_sentences_list(
            text_list=text_list,
            remove_brackets=remove_brackets,
            remove_endings=remove_endings)
        for sentence_num in range(len(sentences_list)):
            sentence = sentences_list[sentence_num]
            for i in range(window_size):
                text += (' '.join(sentence.split()[i:]) + '\n')
        return text[:-1]


In [5]:
import sys

!{sys.executable} -m pip install nltk

In [6]:
from nltk import ngrams

sentence = posts[0]

N = 3
sixgrams = ngrams(sentence, N)

%timeit [''.join(ngram) for ngram in sixgrams]

283 ns ± 46.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [7]:
from itertools import islice, tee
trigrams = zip(*(islice(seq, index, None) for index, seq in enumerate(tee(sentence, N))))
%timeit [''.join(ngram) for ngram in trigrams]

231 ns ± 37.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [8]:
for first, *second in ngrams([''.join(ngram) for ngram in ngrams(sentence, N)], 3):
    print(first, ' => ', second)


Уж  =>  ['Уже', 'же ']
Уже  =>  ['же ', 'е 2']
же   =>  ['е 2', ' 29']
е 2  =>  [' 29', '29 ']
 29  =>  ['29 ', '9 о']
29   =>  ['9 о', ' ок']
9 о  =>  [' ок', 'окт']
 ок  =>  ['окт', 'ктя']
окт  =>  ['ктя', 'тяб']
ктя  =>  ['тяб', 'ябр']
тяб  =>  ['ябр', 'бря']
ябр  =>  ['бря', 'ря ']
бря  =>  ['ря ', 'я в']
ря   =>  ['я в', ' в ']
я в  =>  [' в ', 'в О']
 в   =>  ['в О', ' Ол']
в О  =>  [' Ол', 'Оли']
 Ол  =>  ['Оли', 'лим']
Оли  =>  ['лим', 'имп']
лим  =>  ['имп', 'мпи']
имп  =>  ['мпи', 'пий']
мпи  =>  ['пий', 'ийс']
пий  =>  ['ийс', 'йск']
ийс  =>  ['йск', 'ско']
йск  =>  ['ско', 'ком']
ско  =>  ['ком', 'ом ']
ком  =>  ['ом ', 'м п']
ом   =>  ['м п', ' пр']
м п  =>  [' пр', 'про']
 пр  =>  ['про', 'рой']
про  =>  ['рой', 'ойд']
рой  =>  ['ойд', 'йду']
ойд  =>  ['йду', 'дут']
йду  =>  ['дут', 'ут ']
дут  =>  ['ут ', 'т б']
ут   =>  ['т б', ' бо']
т б  =>  [' бо', 'бои']
 бо  =>  ['бои', 'ои ']
бои  =>  ['ои ', 'и р']
ои   =>  ['и р', ' ро']
и р  =>  [' ро', 'роб']
 ро  =>  ['роб',

ежд  =>  ['жде', 'де ']
жде  =>  ['де ', 'е п']
де   =>  ['е п', ' пр']
е п  =>  [' пр', 'пра']
 пр  =>  ['пра', 'рав']
пра  =>  ['рав', 'ави']
рав  =>  ['ави', 'вил']
ави  =>  ['вил', 'ила']
вил  =>  ['ила', 'ла ']
ила  =>  ['ла ', 'а п']
ла   =>  ['а п', ' пр']
а п  =>  [' пр', 'про']
 пр  =>  ['про', 'рос']
про  =>  ['рос', 'ост']
рос  =>  ['ост', 'сты']
ост  =>  ['сты', 'ты:']
сты  =>  ['ты:', 'ы: ']
ты:  =>  ['ы: ', ': д']
ы:   =>  [': д', ' дв']
: д  =>  [' дв', 'дво']
 дв  =>  ['дво', 'вое']
дво  =>  ['вое', 'ое ']
вое  =>  ['ое ', 'е з']
ое   =>  ['е з', ' за']
е з  =>  [' за', 'зах']
 за  =>  ['зах', 'ахо']
зах  =>  ['ахо', 'ход']
ахо  =>  ['ход', 'одя']
ход  =>  ['одя', 'дят']
одя  =>  ['дят', 'ят ']
дят  =>  ['ят ', 'т н']
ят   =>  ['т н', ' на']
т н  =>  [' на', 'на ']
 на  =>  ['на ', 'а а']
на   =>  ['а а', ' ар']
а а  =>  [' ар', 'аре']
 ар  =>  ['аре', 'рен']
аре  =>  ['рен', 'ену']
рен  =>  ['ену', 'ну,']
ену  =>  ['ну,', 'у, ']
ну,  =>  ['у, ', ', о']
у,   =>  [', о',

er   =>  ['r —', ' — ']
r —  =>  [' — ', '— о']
 —   =>  ['— о', ' он']
— о  =>  [' он', 'они']
 он  =>  ['они', 'ни ']
они  =>  ['ни ', 'и у']
ни   =>  ['и у', ' уж']
и у  =>  [' уж', 'уже']
 уж  =>  ['уже', 'же ']
уже  =>  ['же ', 'е в']
же   =>  ['е в', ' вы']
е в  =>  [' вы', 'выс']
 вы  =>  ['выс', 'ыст']
выс  =>  ['ыст', 'сту']
ыст  =>  ['сту', 'туп']
сту  =>  ['туп', 'упи']
туп  =>  ['упи', 'пил']
упи  =>  ['пил', 'или']
пил  =>  ['или', 'ли ']
или  =>  ['ли ', 'и н']
ли   =>  ['и н', ' на']
и н  =>  [' на', 'на ']
 на  =>  ['на ', 'а ч']
на   =>  ['а ч', ' че']
а ч  =>  [' че', 'чем']
 че  =>  ['чем', 'емп']
чем  =>  ['емп', 'мпи']
емп  =>  ['мпи', 'пио']
мпи  =>  ['пио', 'ион']
пио  =>  ['ион', 'она']
ион  =>  ['она', 'нат']
она  =>  ['нат', 'ате']
нат  =>  ['ате', 'те ']
ате  =>  ['те ', 'е в']
те   =>  ['е в', ' в ']
е в  =>  [' в ', 'в А']
 в   =>  ['в А', ' Ан']
в А  =>  [' Ан', 'Анг']
 Ан  =>  ['Анг', 'нгл']
Анг  =>  ['нгл', 'гли']
нгл  =>  ['гли', 'лии']
гли  =>  ['лии',

и н  =>  [' не', 'нез']
 не  =>  ['нез', 'еза']
нез  =>  ['еза', 'зав']
еза  =>  ['зав', 'ави']
зав  =>  ['ави', 'вис']
ави  =>  ['вис', 'иси']
вис  =>  ['иси', 'сим']
иси  =>  ['сим', 'имы']
сим  =>  ['имы', 'мых']
имы  =>  ['мых', 'ых ']
мых  =>  ['ых ', 'х с']
ых   =>  ['х с', ' сп']
х с  =>  [' сп', 'спа']
 сп  =>  ['спа', 'пар']
спа  =>  ['пар', 'арр']
пар  =>  ['арр', 'рри']
арр  =>  ['рри', 'рин']
рри  =>  ['рин', 'инг']
рин  =>  ['инг', 'нга']
инг  =>  ['нга', 'гах']
нга  =>  ['гах', 'ах.']
гах  =>  ['ах.', 'х. ']
ах.  =>  ['х. ', '. \n']
х.   =>  ['. \n', ' \n\n']
. 
  =>  [' \n\n', '\n\nВ']
 

  =>  ['\n\nВ', '\nВс']


В  =>  ['\nВс', 'Все']

Вс  =>  ['Все', 'сех']
Все  =>  ['сех', 'ех,']
сех  =>  ['ех,', 'х, ']
ех,  =>  ['х, ', ', к']
х,   =>  [', к', ' ко']
, к  =>  [' ко', 'кот']
 ко  =>  ['кот', 'от ']
кот  =>  ['от ', 'т в']
от   =>  ['т в', ' вс']
т в  =>  [' вс', 'вст']
 вс  =>  ['вст', 'ста']
вст  =>  ['ста', 'тан']
ста  =>  ['тан', 'ане']
тан  =>  ['ане', 'нет']
ане 

ора  =>  ['рая', 'ая ']
рая  =>  ['ая ', 'я н']
ая   =>  ['я н', ' на']
я н  =>  [' на', 'на ']
 на  =>  ['на ', 'а с']
на   =>  ['а с', ' се']
а с  =>  [' се', 'сег']
 се  =>  ['сег', 'его']
сег  =>  ['его', 'год']
его  =>  ['год', 'одн']
год  =>  ['одн', 'дня']
одн  =>  ['дня', 'няш']
дня  =>  ['няш', 'яшн']
няш  =>  ['яшн', 'шни']
яшн  =>  ['шни', 'ний']
шни  =>  ['ний', 'ий ']
ний  =>  ['ий ', 'й м']
ий   =>  ['й м', ' мо']
й м  =>  [' мо', 'мом']
 мо  =>  ['мом', 'оме']
мом  =>  ['оме', 'мен']
оме  =>  ['мен', 'ент']
мен  =>  ['ент', 'нт ']
ент  =>  ['нт ', 'т п']
нт   =>  ['т п', ' пр']
т п  =>  [' пр', 'пре']
 пр  =>  ['пре', 'ред']
пре  =>  ['ред', 'едс']
ред  =>  ['едс', 'дст']
едс  =>  ['дст', 'ста']
дст  =>  ['ста', 'тав']
ста  =>  ['тав', 'авл']
тав  =>  ['авл', 'вля']
авл  =>  ['вля', 'ляе']
вля  =>  ['ляе', 'яет']
ляе  =>  ['яет', 'ет ']
яет  =>  ['ет ', 'т с']
ет   =>  ['т с', ' см']
т с  =>  [' см', 'сме']
 см  =>  ['сме', 'мес']
сме  =>  ['мес', 'есь']
мес  =>  ['есь',

...

нан  =>  ['ано', 'нос']
ано  =>  ['нос', 'оси']
нос  =>  ['оси', 'сит']
оси  =>  ['сит', 'ит ']
сит  =>  ['ит ', 'т н']
ит   =>  ['т н', ' не']
т н  =>  [' не', 'нед']
 не  =>  ['нед', 'едо']
нед  =>  ['едо', 'дос']
едо  =>  ['дос', 'ост']
дос  =>  ['ост', 'ста']
ост  =>  ['ста', 'тат']
ста  =>  ['тат', 'ато']
тат  =>  ['ато', 'точ']
ато  =>  ['точ', 'очн']
точ  =>  ['очн', 'чно']
очн  =>  ['чно', 'но ']
чно  =>  ['но ', 'о с']
но   =>  ['о с', ' си']
о с  =>  [' си', 'сил']
 си  =>  ['сил', 'иль']
сил  =>  ['иль', 'льн']
иль  =>  ['льн', 'ьно']
льн  =>  ['ьно', 'ное']
ьно  =>  ['ное', 'ое ']
ное  =>  ['ое ', 'е п']
ое   =>  ['е п', ' по']
е п  =>  [' по', 'пов']
 по  =>  ['пов', 'овр']
пов  =>  ['овр', 'вре']
овр  =>  ['вре', 'реж']
вре  =>  ['реж', 'ежд']
реж  =>  ['ежд', 'жде']
ежд  =>  ['жде', 'ден']
жде  =>  ['ден', 'ени']
ден  =>  ['ени', 'ние']
ени  =>  ['ние', 'ие ']
ние  =>  ['ие ', 'е п']
ие   =>  ['е п', ' пр']
е п  =>  [' пр', 'при']
 пр  =>  ['при', 'ри ']
при  =>  ['ри ',

р-к  =>  ['-ко', 'кол']
-ко  =>  ['кол', 'олё']
кол  =>  ['олё', 'лёс']
олё  =>  ['лёс', 'ёс ']
лёс  =>  ['ёс ', 'с о']
ёс   =>  ['с о', ' от']
с о  =>  [' от', 'от ']
 от  =>  ['от ', 'т г']
от   =>  ['т г', ' ги']
т г  =>  [' ги', 'гир']
 ги  =>  ['гир', 'иро']
гир  =>  ['иро', 'рос']
иро  =>  ['рос', 'оск']
рос  =>  ['оск', 'ску']
оск  =>  ['ску', 'кут']
ску  =>  ['кут', 'уте']
кут  =>  ['уте', 'тер']
уте  =>  ['тер', 'ера']
тер  =>  ['ера', 'ра ']
ера  =>  ['ра ', 'а п']
ра   =>  ['а п', ' по']
а п  =>  [' по', 'по ']
 по  =>  ['по ', 'о 3']
по   =>  ['о 3', ' 35']
о 3  =>  [' 35', '350']
 35  =>  ['350', '50 ']
350  =>  ['50 ', '0 В']
50   =>  ['0 В', ' Вт']
0 В  =>  [' Вт', 'Вт ']
 Вт  =>  ['Вт ', 'т к']
Вт   =>  ['т к', ' ка']
т к  =>  [' ка', 'каж']
 ка  =>  ['каж', 'ажд']
каж  =>  ['ажд', 'ждо']
ажд  =>  ['ждо', 'дое']
ждо  =>  ['дое', 'ое.']
дое  =>  ['ое.', 'е. ']
ое.  =>  ['е. ', '. З']
е.   =>  ['. З', ' За']
. З  =>  [' За', 'За ']
 За  =>  ['За ', 'а п']
За   =>  ['а п',

 .   =>  ['. П', ' Пр']
. П  =>  [' Пр', 'Пре']
 Пр  =>  ['Пре', 'ред']
Пре  =>  ['ред', 'еды']
ред  =>  ['еды', 'дыд']
еды  =>  ['дыд', 'ыду']
дыд  =>  ['ыду', 'дущ']
ыду  =>  ['дущ', 'ущи']
дущ  =>  ['ущи', 'щий']
ущи  =>  ['щий', 'ий ']
щий  =>  ['ий ', 'й р']
ий   =>  ['й р', ' ро']
й р  =>  [' ро', 'роб']
 ро  =>  ['роб', 'обо']
роб  =>  ['обо', 'бот']
обо  =>  ['бот', 'от ']
бот  =>  ['от ', 'т к']
от   =>  ['т к', ' ко']
т к  =>  [' ко', 'ком']
 ко  =>  ['ком', 'ома']
ком  =>  ['ома', 'ман']
ома  =>  ['ман', 'анд']
ман  =>  ['анд', 'нды']
анд  =>  ['нды', 'ды,']
нды  =>  ['ды,', 'ы, ']
ды,  =>  ['ы, ', ', «']
ы,   =>  [', «', ' «К']
, «  =>  [' «К', '«Ка']
 «К  =>  ['«Ка', 'Кат']
«Ка  =>  ['Кат', 'атю']
Кат  =>  ['атю', 'тюш']
атю  =>  ['тюш', 'юша']
тюш  =>  ['юша', 'ша»']
юша  =>  ['ша»', 'а»,']
ша»  =>  ['а»,', '», ']
а»,  =>  ['», ', ', п']
»,   =>  [', п', ' пр']
, п  =>  [' пр', 'при']
 пр  =>  ['при', 'рин']
при  =>  ['рин', 'ини']
рин  =>  ['ини', 'ним']
ини  =>  ['ним',

юща  =>  ['щая', 'аяс']
щая  =>  ['аяс', 'яся']
аяс  =>  ['яся', 'ся ']
яся  =>  ['ся ', 'я б']
ся   =>  ['я б', ' бр']
я б  =>  [' бр', 'бро']
 бр  =>  ['бро', 'рон']
бро  =>  ['рон', 'оня']
рон  =>  ['оня', 'ня,']
оня  =>  ['ня,', 'я, ']
ня,  =>  ['я, ', ', н']
я,   =>  [', н', ' ни']
, н  =>  [' ни', 'низ']
 ни  =>  ['низ', 'изк']
низ  =>  ['изк', 'зки']
изк  =>  ['зки', 'кий']
зки  =>  ['кий', 'ий ']
кий  =>  ['ий ', 'й ц']
ий   =>  ['й ц', ' це']
й ц  =>  [' це', 'цен']
 це  =>  ['цен', 'ент']
цен  =>  ['ент', 'нтр']
ент  =>  ['нтр', 'тр ']
нтр  =>  ['тр ', 'р м']
тр   =>  ['р м', ' ма']
р м  =>  [' ма', 'мас']
 ма  =>  ['мас', 'асс']
мас  =>  ['асс', 'сс,']
асс  =>  ['сс,', 'с, ']
сс,  =>  ['с, ', ', у']
с,   =>  [', у', ' уд']
, у  =>  [' уд', 'уда']
 уд  =>  ['уда', 'дар']
уда  =>  ['дар', 'ар ']
дар  =>  ['ар ', 'р м']
ар   =>  ['р м', ' мо']
р м  =>  [' мо', 'мол']
 мо  =>  ['мол', 'оло']
мол  =>  ['оло', 'лот']
оло  =>  ['лот', 'ото']
лот  =>  ['ото', 'том']
ото  =>  ['том',

  =>  ['т\nО', '\nОр']
т
О  =>  ['\nОр', 'Ору']

Ор  =>  ['Ору', 'руж']
Ору  =>  ['руж', 'ужи']
руж  =>  ['ужи', 'жие']
ужи  =>  ['жие', 'ие:']
жие  =>  ['ие:', 'е: ']
ие:  =>  ['е: ', ': Э']
е:   =>  [': Э', ' Эл']
: Э  =>  [' Эл', 'Эле']
 Эл  =>  ['Эле', 'лек']
Эле  =>  ['лек', 'ект']
лек  =>  ['ект', 'ктр']
ект  =>  ['ктр', 'тро']
ктр  =>  ['тро', 'рон']
тро  =>  ['рон', 'оно']
рон  =>  ['оно', 'нож']
оно  =>  ['нож', 'ожн']
нож  =>  ['ожн', 'жни']
ожн  =>  ['жни', 'ниц']
жни  =>  ['ниц', 'ицы']
ниц  =>  ['ицы', 'цы ']
ицы  =>  ['цы ', 'ы п']
цы   =>  ['ы п', ' по']
ы п  =>  [' по', 'по ']
 по  =>  ['по ', 'о м']
по   =>  ['о м', ' ме']
о м  =>  [' ме', 'мет']
 ме  =>  ['мет', 'ета']
мет  =>  ['ета', 'тал']
ета  =>  ['тал', 'алл']
тал  =>  ['алл', 'ллу']
алл  =>  ['ллу', 'лу ']
ллу  =>  ['лу ', 'у с']
лу   =>  ['у с', ' с ']
у с  =>  [' с ', 'с у']
 с   =>  ['с у', ' ус']
с у  =>  [' ус', 'уси']
 ус  =>  ['уси', 'сил']
уси  =>  ['сил', 'или']
сил  =>  ['или', 'лие']
или  =>  ['лие',

In [34]:
alphabet = ''.join([chr(i) for i in range(ord('а'), ord('я') + 1)]) + 'ё '
d = {char: i for i, char in enumerate(alphabet)}
d

{'а': 0,
 'б': 1,
 'в': 2,
 'г': 3,
 'д': 4,
 'е': 5,
 'ж': 6,
 'з': 7,
 'и': 8,
 'й': 9,
 'к': 10,
 'л': 11,
 'м': 12,
 'н': 13,
 'о': 14,
 'п': 15,
 'р': 16,
 'с': 17,
 'т': 18,
 'у': 19,
 'ф': 20,
 'х': 21,
 'ц': 22,
 'ч': 23,
 'ш': 24,
 'щ': 25,
 'ъ': 26,
 'ы': 27,
 'ь': 28,
 'э': 29,
 'ю': 30,
 'я': 31,
 'ё': 32,
 ' ': 33}

In [61]:
import itertools
len(list(itertools.permutations(alphabet, 2))), len(alphabet), 34 * 34, (' ', ' ') in l

(1122, 34, 1156, False)

In [63]:
from more_itertools import distinct_permutations
l = list(distinct_permutations(alphabet, 2))
len(l), len(alphabet), 34 * 34 , (' ', ' ') in l

(1122, 34, 1156, False)

In [66]:
p = list(itertools.product(alphabet, repeat=2))
len(p), len(alphabet), 34 * 34, (' ', ' ') in p

(1156, 34, 1156, True)

In [71]:
from itertools import product

def encode_ngrams_dicts(alphabet: str, n: int) -> list:
    ngrams_list = [''.join(ngram) for ngram in product(alphabet, repeat=n)]
    ngram2int, int2ngram = {}, {}
    for i, ngram in enumerate(ngrams_list):
        ngram2int[ngram] = i
        int2ngram[i] = ngram
    return ngram2int, int2ngram

In [77]:
from itertools import product
from nltk import ngrams


class NgramEncoder:
    
    alpabet: str
    ngram2int: dict
    int2ngram: dict
    n_gramm_state: int
        
    def __init__(self, alphabet: str, n_gramm_state: int):
        self.alphabet = alphabet
        self.n_gramm_state = n_gramm_state
        
        ngrams_list = [''.join(ngram) for ngram in product(alphabet, repeat=n_gramm_state)]
        ngram2int, int2ngram = {}, {}
        for i, ngram in enumerate(ngrams_list):
            ngram2int[ngram] = i
            int2ngram[i] = ngram
        self.ngram2int = ngram2int
        self.int2ngram = int2ngram
    
    def encode_ngrams_list(self, ngrams_list: list) -> tuple:
        return tuple(self.ngram2int[ngram] for ngram in ngrams_list)    

In [78]:
RU_ALPHABET = ''.join([chr(i) for i in range(ord('а'), ord('я') + 1)]) + 'ё'
EN_ALPHABET = ''.join([chr(i) for i in range(ord('a'), ord('z') + 1)])

class MarkovModel:
    
    model: dict
    n_gramm_state: int
    model_state: int
    encoder: NgramEncoder
    tokenizer = Tokenizer()
    
    def __init__(self, model: dict, encoder: NgramEncoder, n_gramm_state=2, model_state=3):
        self.model = model
        self.n_gramm_state = n_gramm_state
        self.model_state = model_state
        
    def __split_to_sentencies(self, text: str) -> list:
        return self.tokenizer.tokenize(text)
    
    def __split_to_n_grams(self, word: str, n: int) -> list:
        return ngrams(word, n)
    
    @classmethod
    def train(cls, 
              text_corpus: str, 
              n_gramm_state=2, 
              model_state=2, 
              alphabet=RU_ALPHABET + EN_ALPHABET + ' '):
        model = dict()
        encoder = NgramEncoder(
            alphabet=alphabet, 
            n_gramm_state=n_gramm_state)
        for sentence in cls.__split_to_sentencies(cls, text_corpus):
            for key, *ngrams_list in ngrams([''.join(ngram) for ngram in cls.__split_to_n_grams(cls, sentence, n=n_gramm_state)], model_state + 1):
                ngrams_tuple = encoder.encode_ngrams_list(ngrams_list)
                encode_ngram = encoder.ngram2int[key]
                if encode_ngram in model:
                    if ngrams_tuple in model[encode_ngram]:
                        model[encode_ngram][ngrams_tuple] += 1
                    else:
                        model[encode_ngram][ngrams_tuple] = 1
                else:
                    model[encode_ngram] = {
                        ngrams_tuple: 1
                    }
        return cls(
            model=model,
            encoder=encoder,
            n_gramm_state=n_gramm_state,
            model_state=model_state)
    
    def make_word() -> str:
        pass                     

In [79]:
model = MarkovModel.train(sentence)
model.model

{1146: {(365, 359): 7,
  (368, 485): 28,
  (368, 511): 3,
  (377, 1030): 4,
  (360, 17): 1,
  (365, 317): 1},
 365: {(359, 3599): 2,
  (313, 797): 2,
  (359, 3552): 2,
  (359, 3559): 1,
  (359, 3542): 1,
  (313, 788): 6,
  (359, 3554): 1,
  (311, 665): 1,
  (311, 687): 4,
  (313, 785): 3,
  (359, 3557): 2,
  (311, 674): 3,
  (359, 3553): 1,
  (309, 552): 1,
  (359, 3541): 1,
  (318, 1139): 3,
  (317, 1038): 1,
  (317, 1030): 1,
  (313, 793): 1},
 359: {(3599, 3554): 1,
  (3542, 136): 2,
  (3555, 911): 1,
  (3555, 914): 6,
  (3555, 916): 3,
  (3547, 420): 4,
  (3557, 1033): 1,
  (3541, 74): 6,
  (3542, 125): 2,
  (3550, 614): 7,
  (3599, 3555): 2,
  (3552, 725): 6,
  (3559, 1163): 2,
  (3542, 147): 2,
  (3542, 179): 7,
  (3548, 493): 1,
  (3547, 436): 1,
  (3557, 1020): 1,
  (3556, 974): 6,
  (3557, 1035): 2,
  (3554, 858): 3,
  (3560, 1211): 3,
  (3557, 1038): 33,
  (3599, 3543): 1,
  (3558, 1094): 3,
  (3557, 1027): 1,
  (3544, 248): 2,
  (3588, 2931): 1,
  (3548, 539): 5,
  (3550, 60